# Sensitivities computation via automated adjoint for a 2D shallow water model

This notebook demonstrates how to compute the sensitivities (gradient) of a cost function with respect to control parameters using automated adjoints, a tool provided by the Firedrake library. The model used in this notebook is a shallow water model on a sphere. 

## Cost function and model
The cost function measures the summation of the kinetic and the potential energy of the fluid, which is modelled by the following expression:
\begin{equation}
J = \int_{\Omega} \left( \frac{1}{2} \textbf{u}(\mathbf{x}, t_f) \cdot \textbf{u}(\mathbf{x}, t_f) + \frac{1}{2} g D^2(\mathbf{x}, t_f) \right) \, dx,
\end{equation}
where $\textbf{u} (\mathbf{x}, t_f)$ is the velocity of the fluid at the final step $t_f$, $D(\mathbf{x}, t_f)$ is the fluid depth at the final step, and $g$ is the gravitational acceleration. The model is governed by the following set of equations:

\begin{align}
    \textbf{u}_t + (\textbf{u} \cdot \nabla) \textbf{u} + f \textbf{u}^{\perp} + g \nabla (D + b)  &= 0, \tag{2}\\
    D_t + \nabla \cdot (D \textbf{u}) &= 0, \tag{3}
\end{align}
where $f$ is the Coriolis parameter, the fluid depth is given by $D = H+h-b$, $H$ is the mean fluid depth, $h$ is the free surface height and $b$ is the topography. No boundary conditions are required as we are solving on a spherical domain, $\Omega$.

## Solve the model with Gusto
We first import the libraries required for the computations: ``gusto`` and ``firedrake``.

In [1]:
from firedrake import *
from gusto import *

INFO     Running /Users/JS1075/firedrake_dec24/lib/python3.12/site-packages/ipykernel_launcher.py -f /private/var/folders/f0/llvlmlb50qg6mmlxs8m6d6d00000gp/T/tmpxgk65e8m.json --HistoryManager.hist_file=:memory:


We then define the shallow water parameter $H$ (The fluid depth).

We will use one of the spherical meshes provided by Firedrake: the ``IcosahedralSphereMesh``. As the spherical domain we are solving over is the Earth we specify the radius as 6371220m. The refinement level, ``ref_level``, specifies the number of times the base icosahedron is refined. The argument ``degree`` specifies the polynomial degree of the function space used to represent the coordinates.

In [2]:
R = 6371220.
mesh = IcosahedralSphereMesh(radius=R, refinement_level=3, degree=2)

In [3]:
H = 5960.0
parameters = ShallowWaterParameters(mesh, H=H)

A domain is created with the ``Domain`` Gusto object, which holds the mesh and the function spaces defined on it. It also holds the model’s time interval.

In [4]:
dt = 900.
domain = Domain(mesh, dt, 'BDM', 1)

We can now set up the finite element form of the shallow water equations by passing ``ShallowWaterEquations`` Gusto class the following arguments: ``domain``, ``parameters``,  the expressions ``fexpr`` for the Coriolis parameter and ``bexpr`` for the bottom surface of the fluid.

In [5]:
x = SpatialCoordinate(mesh)
Omega = parameters.Omega
fexpr = 2*Omega*x[2]/R
lamda, theta, _ = lonlatr_from_xyz(x[0], x[1], x[2])
R0 = pi/9.
R0sq = R0**2
lamda_c = -pi/2.
lsq = (lamda - lamda_c)**2
theta_c = pi/6.
thsq = (theta - theta_c)**2
rsq = min_value(R0sq, lsq+thsq)
r = sqrt(rsq)
topog_expr = 2000 * (1 - r/R0)
eqn = ShallowWaterEquations(domain, parameters, fexpr=fexpr, topog_expr=topog_expr)

We specify instructions regarding the output using the ``OutputParameters`` class. The directory name, ``dirname``, must be specified. To prevent losing hard-earned simulation data, Gusto will not allow existing files to be overwritten. Hence, if one wishes to re-run a simulation with the same output filename, the existing results file needs to be moved or deleted first. This is also the place to specify the output frequency (in timesteps) of VTU files. The default is ``dumpfreq=1``, which outputs VTU files at every timestep (very useful when first setting up a problem!). Below we set ``dumpfreq=5``.

We can specify which diagnostics to record over a simulation. The list of available diagnostics in the gusto source code: [diagnostics](https://github.com/firedrakeproject/gusto/blob/main/gusto/diagnostics/diagnostics.py). Since this flow should be in a steady state, it is also instructive to output the steady state error fields for both $\textbf{u}$ and $D$ as this will tell us how close the simulation is to be correct. The errors should not grow in time. They should reduce as the mesh and timestep are refined. We pass these diagnostics into the ``IO`` class, which controls the input and output and stores the fields which will be updated at each timestep.

In [6]:
output = OutputParameters(dirname="adjoint_sw", log_courant=False)
io = IO(domain, output)

We now start to build a setup for time discretisation. We use the ``SemiImplicitQuasiNewton`` approach, which splits the equation into `transport` terms and 'forcing' terms (i.e. everything that does not transport) and solves each separately. That allows for different time-steppers used to transport the velocity and depth fields. We are employing an Implicit Midpoint method for the velocity and an explicit strong stability preserving RK3 (SSPRK3) method for the depth. Since the Courant number for a stable SSPRK3 scheme is lower than that for the Implicit Midpoint method, we do two subcycles of the SSPRK3 scheme per timestep, allowing us to use a longer timestep overall. The full list of time-stepping methods is available at the [time discretisation](https://github.com/firedrakeproject/gusto/blob/main/gusto/time_discretisation.py) python file.

In [7]:
# Transport schemes
transported_fields = [TrapeziumRule(domain, "u"), SSPRK3(domain, "D")]
transport_methods = [DGUpwind(eqn, "u"), DGUpwind(eqn, "D")]

# Time stepper
stepper = SemiImplicitQuasiNewton(eqn, io, transported_fields, transport_methods)

INFO     Physical parameters that take non-default values:


INFO     Omega: 7.292e-05, g: 9.80616, H: 5960.0


We are now ready to specify the initial conditions:
\begin{align}
    \textbf{u}_0 &= \frac{u_{max}}{R} [-y,x,0], \tag{4}\\
    D_0 &= H - \frac{\Omega u_{max} z^2}{g R} \tag{5}
\end{align}

In [8]:
u0 = stepper.fields('u')
D0 = stepper.fields('D')
u_max = 20.   # Maximum amplitude of the zonal wind (m/s)
uexpr = as_vector([-u_max*x[1]/R, u_max*x[0]/R, 0.0])
g = parameters.g  # acceleration due to gravity (m/s^2)
Rsq = R**2
Dexpr = H - ((R * Omega * u_max + 0.5*u_max**2)*x[2]**2/Rsq)/g - bexpr

u0.project(uexpr)
D0.interpolate(Dexpr)

NameError: name 'bexpr' is not defined

When using the `SemiImplicitQuasiNewton` time ``stepper``, we also have to set up any non-zero reference profiles.

In [9]:
Dbar = Function(D0.function_space()).assign(H)
stepper.set_reference_profiles([('D', Dbar)])

We are almost ready to run the model. Before we do so, we have to tape the model to compute the sensitivities via automated adjoint. Hence, at this point, we import ``firedrake.adjoint`` and enable the tape of the forward solver (Shallow Water model) with ``continue_annotation()``.

In [10]:
from firedrake.adjoint import *
continue_annotation()

True

We are finally ready to run the model!

In [11]:
timesteps = 5
stepper.run(0., timesteps*dt)

INFO     


INFO     ========================================


INFO     at start of timestep 1, t=0.0, dt=900.0


INFO     Semi-implicit Quasi Newton: Explicit forcing


INFO     Semi-implicit Quasi Newton: Transport 0: u


...

INFO     Semi-implicit Quasi Newton: Implicit forcing (1, 0)


INFO     Semi-implicit Quasi Newton: Mixed solve (1, 0)


INFO     Semi-implicit Quasi Newton: Implicit forcing (1, 1)


INFO     Semi-implicit Quasi Newton: Mixed solve (1, 1)


INFO     TIMELOOP complete. t=4500.00000, tmax=4500.00000


Next, the final solution of the shallow water solver is used to compute the cost functional (1) as shown in the following cell code.

In [12]:
u_tf = stepper.fields('u')  # Final velocity field
D_tf = stepper.fields('D')  # Final depth field

J = assemble(0.5*inner(u_tf, u_tf)*dx + 0.5*g*D_tf**2*dx)

We define the control variable: initial fluid depth $D_0$ and the velocity $\textbf{u}_0$. Finally, we run the sensitivity computation: gradient of the cost function with respect $D_0$ and $\textbf{u}_0$. The gradient computation is achieved with the ``compute_gradient`` function from the `firedrake.adjoint` module.

In [13]:
control = [Control(D0), Control(u0)]
grad = compute_gradient(J, control)

You can execute a gradient verification with second order [Taylor's test](https://www.dolfinadjoint.org/en/latest/documentation/verification.html). To do so, we define the reduced functional as follows:

In [14]:
J_hat = ReducedFunctional(J, control)

We then verify the gradient computation using the ``taylor_test``. We already taped the forward model. Any additional operation as below, for  Taylor's test, does not require the tape to be enabled. Thus, we can stop annotating.

In [15]:
import numpy as np
with stop_annotating():
    # Stop annotation to perform the Taylor test
    h0 = Function(D0.function_space())
    h1 = Function(u0.function_space())
    h0.assign(D0 * np.random.rand())
    h1.assign(u0 * np.random.rand())
    taylor_test(J_hat, [D0, u0], [h0, h1])

Running Taylor test


Computed residuals: [1423492808354.25, 355873202081.125, 88968300519.5625, 22242075114.78125]
Computed convergence rates: [np.float64(2.0000000000301514), np.float64(2.0000000000116556), np.float64(2.0000000009800445)]


That is great! We have a sensitivity computation via automated adjoint with Gusto! In addition, we have verified the gradient computation with a second-order Taylor's test, which is a good practice to ensure the correctness of the gradient computation. See the results above are close to the expected values, 2.0.

For a notebook demonstration of adjoints, it is good practice to clear the tape. This is because if we do not clear the tape, the tape will keep growing every time we run the model.

In [16]:
tape = get_working_tape()
tape.clear_tape()